In [1]:
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import MaaSSim

In [2]:
#!pip install MaaSSim
import MaaSSim.utils
import logging
from dotmap import DotMap
import MaaSSim.simulators
from MaaSSim.data_structures import structures as inData
from MaaSSim.acceptance import ResultS

In [3]:
import osmnx as ox
import networkx as nx
import random
import pandas as pd
import matplotlib.pyplot as plt          
import numpy as np
import seaborn as sns

In [4]:
# Do not forget to set the radius for centeral nodes calculation
# The minimum trip distance is considered 1000m and this can be modified through utils.py: generate_demand function. 
# long attribute should be calculated according to the size of city
# Platform fare is 2$ per km
# Passengers' and drivers' positions are fixed.

params = MaaSSim.utils.get_config('glance.json')
params.use_fixed_postions = True
params.times.patience = 600 #in second
params.dist_threshold = 10000 #in meter
params.vehicle_fixed_positions= True  #this should be fixed
params.city = "Delft, Netherlands"
params.paths.G = "../../data/graphs/Delft.graphml"
params.paths.skim = "../../data/graphs/Delft.csv"
# params.city = "Dayton, USA"
# params.paths.G = "../../data/graphs/Dayton.graphml"
# params.paths.skim = "../../data/graphs/Dayton.csv"
params.paths.ride_skim = "../../data/graphs/Delft_ride.csv"
params.nP = 500
params.nV = 20
params.simTime = 5

In [5]:
sim= MaaSSim.simulators.simulate(params=params)

07-12-21 18:19:42-WARNING-Setting up 5h simulation at 2021-12-07 15:50:10 for 20 vehicles and 500 passengers in Delft, Netherlands
07-12-21 18:19:42-INFO--------------------	Starting simulation	-------------------
07-12-21 18:19:43-INFO-pax    2  got to dest at                           2021-12-07 15:55:18
07-12-21 18:19:43-INFO-pax    1  got to dest at                           2021-12-07 15:56:09
07-12-21 18:19:43-INFO-pax    3  got to dest at                           2021-12-07 15:57:38
07-12-21 18:19:43-INFO-pax    8  got to dest at                           2021-12-07 15:58:46
07-12-21 18:19:43-INFO-pax    6  got to dest at                           2021-12-07 15:59:35
07-12-21 18:19:43-INFO-pax   10  got to dest at                           2021-12-07 16:00:21
07-12-21 18:19:43-INFO-pax    7  got to dest at                           2021-12-07 16:00:46
07-12-21 18:19:43-INFO-pax    5  got to dest at                           2021-12-07 16:01:43
07-12-21 18:19:43-INFO-pax    9  g

In [6]:
r_def= ResultS(sim)

In [7]:
r_def.passengers['waiting_t[sec]'].mean()

241.67

In [8]:
def my_function(**kwargs): # user defined function to represent agent decisions
    if random.uniform(0, 1) > 0.7:
        return True
    else:
        return False
    
sim = MaaSSim.simulators.simulate(params=params, f_driver_decline=my_function) 
# run the simulation from a given configuration

07-12-21 18:14:19-WARNING-Setting up 5h simulation at 2021-12-07 15:44:50 for 20 vehicles and 500 passengers in Delft, Netherlands
07-12-21 18:14:20-INFO--------------------	Starting simulation	-------------------
07-12-21 18:14:20-INFO-pax    1  got to dest at                           2021-12-07 15:51:43
07-12-21 18:14:20-WARNING-pax    8  got rejected by vehicle 20               2021-12-07 15:51:56
07-12-21 18:14:20-WARNING-pax    8  got rejected by vehicle 5                2021-12-07 15:51:56
07-12-21 18:14:20-WARNING-pax    8  got rejected by vehicle 12               2021-12-07 15:51:56
07-12-21 18:14:20-WARNING-pax    8  got rejected by vehicle 10               2021-12-07 15:51:56
07-12-21 18:14:20-WARNING-pax    9  got rejected by vehicle 3                2021-12-07 15:52:27
07-12-21 18:14:20-INFO-pax    3  got to dest at                           2021-12-07 15:53:13
07-12-21 18:14:20-WARNING-pax   12  got rejected by vehicle 18               2021-12-07 15:53:32
07-12-21 18:14:2

In [9]:
r_def= ResultS(sim)
r_def.passengers['waiting_t[sec]'].mean()

317.792

In [13]:
# Homogeneous Primary BIP implemented on First regime 

import pandas as pd
import math
import random as random
from dotmap import DotMap


def f_decline(veh, **kwargs):
    
    df = pd.DataFrame(veh.myrides)
    ASC = 1.5   #ASC
    
    print('hi')
    working_shift = sim.params.simTime*3600 - veh.veh['shift_start']                               #Time1_loc
    T1 = int(working_shift/3)
    request_time = df[df['event']=='RECEIVES_REQUEST'].iloc[-1]['t']
    
    if  request_time in range(veh.veh['shift_start'], veh.veh['shift_start']+T1):
        Time1 = 1
    else:
        Time1 = 0
        
    if veh.veh['pos'] in sim.inData.stats.central_nodes:
        loc = 1
    else:
        loc = 0
        
        
    d = veh.offers[0]['request']["origin"]                                                       #pickup_time
    o = veh.veh.pos
    pickup_time = veh.sim.skims.ride[o][d]/60  #minutes
      
    t = df[df['event']=='RECEIVES_REQUEST'].iloc[-1]['t']                                        #waiting_time
    
    if 'ARRIVES_AT_DROPOFF' in df['event'].unique():
        t0 = df[df['event']=='ARRIVES_AT_DROPOFF'].iloc[-1]['t']
    else:
        t0 = df[df['event']=='OPENS_APP'].iloc[-1]['t']
    waiting_time = (t - t0)/60 #minutes 
    
    surge_price = 0                                                                               #surge_price
    
    req = 1                         #req                                                          #req_long_rate_dec
    
    if (veh.offers[0]["request"]["dist"]/sim.params.speeds.ride)/60 > 6.5: #long
        long = 1
    else:
        long = 0
        
    rate = sim.pax[veh.offers[0]['pax_id']].pax.get('rate',5)   #rate
    
    if len(veh.declines.index) == 0:          #dec
        last_declined = 'False'
    else:
        last_declined = veh.declines.loc[len(veh.declines.index)-1]['declined']
        
    if last_declined == 'True':
        dec = 1
    else:
        dec = 0
        
    
           
    V = ((ASC*1) + (Time1*loc*(-0.265)) + (pickup_time*(-0.0491)) + (waiting_time*(-0.0173)) + 
        ((req*long*rate*dec)*0.0909) + (surge_price*0.099))

    #eps = ranres[veh.id-1]
    #V = ((ASC*1) + (Time1*loc*(-0.265)) + (pickup_time*(-0.0491)) + (waiting_time*(-0.0173)) + 
    #    ((req*long*rate*dec)*0.0909) + (surge_price*0.099)) + eps    
    
    acc_prob = (math.exp(V))/(1+math.exp(V))
    
#     print('----------------------------------------------------------------------------')
#     print('vehicle id is ', veh.id)
#     print('acceptance probability is ', acc_prob)
#     print('ASC*1 is ', ASC*1)
#     print('Time1*loc*(-0.263) is ',Time1*loc*(-0.263))
#     print('pickup_time*(-0.0491) is ',pickup_time*(-0.0491))   
#     print('waiting_time*(-0.0173) is ',waiting_time*(-0.0173))
#     print('(req*long*rate*dec)*0.0909 is ',(req*long*rate*dec)*0.0909)
#     print('surge_price*0.099 is ',surge_price*0.099)
#     #print('Random residual is ', eps)
#     print('Systematic Utility V is ', V)
#     print('----------------------------------------------------------------------------')
    

    
    #if acc_prob > 0.5:
    if acc_prob > random.uniform(0, 1):
        return False
    else:
        return True

sim.make_and_run(f_driver_decline=f_decline)

07-12-21 18:21:37-INFO--------------------	Starting simulation	-------------------
hi
07-12-21 18:21:37-WARNING-pax    0  got rejected by vehicle 1                2021-12-07 15:50:10
07-12-21 18:21:37-WARNING-Nobody likes each other, Qs 1veh; 1req; tabu 2
hi
07-12-21 18:21:37-WARNING-pax    0  got rejected by vehicle 2                2021-12-07 15:50:10
07-12-21 18:21:37-WARNING-Nobody likes each other, Qs 2veh; 1req; tabu 3
hi
07-12-21 18:21:37-WARNING-pax    0  got rejected by vehicle 3                2021-12-07 15:50:10
07-12-21 18:21:37-WARNING-Nobody likes each other, Qs 3veh; 1req; tabu 4
hi
07-12-21 18:21:37-WARNING-pax    0  got rejected by vehicle 4                2021-12-07 15:50:10
07-12-21 18:21:37-WARNING-Nobody likes each other, Qs 4veh; 1req; tabu 5
hi
07-12-21 18:21:37-WARNING-pax    0  got rejected by vehicle 5                2021-12-07 15:50:10
07-12-21 18:21:37-WARNING-Nobody likes each other, Qs 5veh; 1req; tabu 6
hi
07-12-21 18:21:37-WARNING-pax    0  got rejected 

In [12]:
r_def= ResultS(sim)
r_def.passengers['waiting_t[sec]'].mean()

193.222

In [ ]:
def f_decline(veh, **kwargs):
    
    if veh.id < 11:
        print('in')
        if random.uniform(0, 1) > ave_p:
            return True
        else:
            return False
    else:
        df = pd.DataFrame(veh.myrides)
        ASC = 1.5                                                                                   #ASC

        working_shift = sim.params.simTime*3600 - veh.veh['shift_start']                               #Time1_loc
        T1 = int(working_shift/3)
        request_time = df[df['event']=='RECEIVES_REQUEST'].iloc[-1]['t']

        if  request_time in range(veh.veh['shift_start'], veh.veh['shift_start']+T1):
            Time1 = 1
        else:
            Time1 = 0

        if veh.veh['pos'] in sim.inData.stats.central_nodes:
            loc = 1
        else:
            loc = 0


        d = veh.offers[0]['request']["origin"]                                                       #pickup_time
        o = veh.veh.pos
        pickup_time = veh.sim.skims.ride[o][d]/60  #minutes

        t = df[df['event']=='RECEIVES_REQUEST'].iloc[-1]['t']                                        #waiting_time

        if 'ARRIVES_AT_DROPOFF' in df['event'].unique():
            t0 = df[df['event']=='ARRIVES_AT_DROPOFF'].iloc[-1]['t']
        else:
            t0 = df[df['event']=='OPENS_APP'].iloc[-1]['t']
        waiting_time = (t - t0)/60 #minutes 

        surge_price = 0                                                                               #surge_price

        req = 1                         #req                                                          #req_long_rate_dec

        if (veh.offers[0]["request"]["dist"]/sim.params.speeds.ride)/60 > 6.5: #long
            long = 1
        else:
            long = 0

        rate = sim.pax[veh.offers[0]['pax_id']].pax.get('rate',5)   #rate

        if len(veh.declines.index) == 0:          #dec
            last_declined = 'False'
        else:
            last_declined = veh.declines.loc[len(veh.declines.index)-1]['declined']

        if last_declined == 'True':
            dec = 1
        else:
            dec = 0


        V = ((ASC*1) + (Time1*loc*(-0.265)) + (pickup_time*(-0.0491)) + (waiting_time*(-0.0173)) + 
            ((req*long*rate*dec)*0.0909) + (surge_price*0.099))


        acc_prob = (math.exp(V))/(1+math.exp(V))

        if acc_prob > random.uniform(0, 1):
            return False
        else:
            return True
                
sim.make_and_run(f_drivd_a_decline=f_decline)

07-12-21 18:24:03-INFO--------------------	Starting simulation	-------------------
hi
07-12-21 18:24:03-WARNING-pax    0  got rejected by vehicle 1                2021-12-07 15:50:10
07-12-21 18:24:03-WARNING-Nobody likes each other, Qs 1veh; 1req; tabu 2
hi
07-12-21 18:24:03-WARNING-pax    0  got rejected by vehicle 2                2021-12-07 15:50:10
07-12-21 18:24:03-WARNING-Nobody likes each other, Qs 2veh; 1req; tabu 3
hi
07-12-21 18:24:03-WARNING-pax    0  got rejected by vehicle 3                2021-12-07 15:50:10
07-12-21 18:24:03-WARNING-Nobody likes each other, Qs 3veh; 1req; tabu 4
hi
07-12-21 18:24:03-WARNING-pax    0  got rejected by vehicle 4                2021-12-07 15:50:10
07-12-21 18:24:03-WARNING-Nobody likes each other, Qs 4veh; 1req; tabu 5
hi
07-12-21 18:24:03-WARNING-pax    0  got rejected by vehicle 5                2021-12-07 15:50:10
07-12-21 18:24:03-WARNING-Nobody likes each other, Qs 5veh; 1req; tabu 6
hi
07-12-21 18:24:03-WARNING-pax    0  got rejected 

In [9]:
r = ResultS(sim)

In [10]:
r.passengers['waiting_t[sec]'].mean()

241.67